In [23]:
import numpy as np
import matplotlib.pyplot as plt
import torch
import pandas as pd
import time
import wandb
import pytorch_lightning as pl

In [24]:
import FunctionCollection as fc
import importlib
fc = importlib.reload(fc)

In [25]:
def AzZe_target_constructor(dataset, transformer):
    az_ze = dataset.data.y.view(-1,10,1)[:,[8,9]]
    tfs = transformer
    
    az = torch.tensor(tfs['truth']['azimuth'].inverse_transform(az_ze[:,0]),dtype=torch.float)
    ze = torch.tensor(tfs['truth']['zenith'].inverse_transform(az_ze[:,1]),dtype=torch.float) #the range seems to be about [0,pi/2]?. Makes sense, Muons come from the atmosphere
    
    dataset.data.y = torch.cat([az,ze],dim=1).flatten()
    dataset.slices['y'] = np.arange(0,len(dataset.data.y)+1, 2)
    return dataset

In [26]:
path = "C:/Users/jv97/Desktop/github/Neutrino-Machine-Learning/datasets"
filename = "muon_100k_set11_SRT.pt"
transformer = pd.read_pickle(path + "/transformers.pkl")

batch_size = 512

dataset, train_loader, test_loader, val_loader = fc.dataset_preparator(name = filename,
                                                                       path = path,
                                                                       transformer = transformer,
                                                                       tc = AzZe_target_constructor,
                                                                       fc = fc.custom_feature_constructor,
                                                                       shuffle = True,
                                                                       TrTV_split = (0.9,0.9,1),
                                                                       batch_size = batch_size)

2021-02-19 22:24:12.182999: loading data..
2021-02-19 22:24:12.430065: executing target constructor..
2021-02-19 22:24:12.434065: executing feature constructor..
2021-02-19 22:24:21.501501: shuffling dataset..
2021-02-19 22:24:21.531492: defining dataloaders..
2021-02-19 22:24:21.532493: Done!


In [27]:
import Model_tmp as M
M = importlib.reload(M)

args = {'N_edge_feats': 6,
        'N_dom_feats': 5,
        'N_targets': 3,
        'N_metalayers': 5,
        'N_hcs': 16,
        'wandb_activated': True,
        'lr': 1e-3,
        'batch_size': batch_size}

model = M.Load_model('Spherical_NLLH',args)

# if args['wandb_activated']:
#     wandb.login()
#     wandb.init(name='Test run - Delete me',
#               project='Neutrino-Machine-Learning',
#               notes='')
#     wandb.config.lr = args['lr']
#     wandb.config.batch_size = args['batch_size']
#     wandb.config.scheduler = 'epoch_warmup=2,max=25,a=0.05,q=0.2'
#     wandb.watch(model)

Remember all accuracies are positive and defined to go towards 0 in the optimal case.


In [28]:
early_stop_callback = pl.callbacks.early_stopping.EarlyStopping(monitor='Val Acc', min_delta=0.00, patience=3, verbose=False, mode='min')

from pytorch_lightning.loggers import WandbLogger
wandb_logger = WandbLogger(name='Test run - delete me',
                           project='Neutrino-Machine-Learning')

trainer = pl.Trainer(gpus=1, #-1 for all gpus
                     max_epochs=1, 
                     callbacks=[early_stop_callback], 
                     logger = wandb_logger if args['wandb_activated'] else False)

if args['wandb_activated']:
    wandb_logger.watch(model)

GPU available: True, used: True
TPU available: None, using: 0 TPU cores


In [31]:
trainer.fit(model, train_loader, val_loader)


  | Name              | Type       | Params
-------------------------------------------------
0 | act               | SiLU       | 0     
1 | x_encoder         | Linear     | 480   
2 | edge_attr_encoder | Linear     | 112   
3 | u_encoder         | Linear     | 1.9 K 
4 | ops               | ModuleList | 399 K 
5 | decoders          | ModuleList | 75.4 K
6 | decoder           | Linear     | 99    
-------------------------------------------------
477 K     Trainable params
0         Non-trainable params
477 K     Total params
1.908     Total estimated model params size (MB)


wandb: WARNING Step must only increase in log calls.  Step 175 < 674; dropping {'val_batch_acc_step/epoch_0': 0.01317131519317627}.
wandb: WARNING Step must only increase in log calls.  Step 176 < 674; dropping {'val_batch_acc_step/epoch_0': 0.0308382511138916}.
wandb: WARNING Step must only increase in log calls.  Step 177 < 674; dropping {'val_batch_acc_step/epoch_0': 0.037039756774902344}.
wandb: WARNING Step must only increase in log calls.  Step 178 < 674; dropping {'val_batch_acc_step/epoch_0': 0.0026885271072387695}.
wandb: WARNING Step must only increase in log calls.  Step 179 < 674; dropping {'val_batch_acc_step/epoch_0': 0.01820605993270874}.
wandb: WARNING Step must only increase in log calls.  Step 180 < 674; dropping {'val_batch_acc_step/epoch_0': 0.021343231201171875}.
wandb: WARNING Step must only increase in log calls.  Step 181 < 674; dropping {'val_batch_acc_step/epoch_0': 0.037691354751586914}.
wandb: WARNING Step must only increase in log calls.  Step 182 < 674; dr

1

In [32]:
wandb.finish()

Train Loss,-64.21049
Train Acc,0.03901
epoch,0
_runtime,354
_timestamp,1613770258
_step,702
val_batch_acc_epoch,0.02508
Val Acc,0.02508


Train Loss,█████▅▆▅▄▁
Train Acc,██▃▂▂▁▂▂▁▂
epoch,▁▁▁▁▁▁▁▁▁▁▁▁▁
_runtime,▁▂▂▃▄▄▄▄▆▆▇▇█
_timestamp,▁▂▂▃▄▄▄▄▆▆▇▇█
_step,▁▂▂▂▄▅▆▆▆▇▇██
val_batch_acc_epoch,█▃▁
Val Acc,█▃▁


In [22]:
help(model.logger.log_metrics)

Help on method log_metrics in module pytorch_lightning.loggers.wandb:

log_metrics(metrics: Dict[str, float], step: Union[int, NoneType] = None) -> None method of pytorch_lightning.loggers.wandb.WandbLogger instance
    Records metrics.
    This method logs metrics as as soon as it received them. If you want to aggregate
    metrics for one specific `step`, use the
    :meth:`~pytorch_lightning.loggers.base.LightningLoggerBase.agg_and_log_metrics` method.
    
    Args:
        metrics: Dictionary with metric names as keys and measured quantities as values
        step: Step number at which the metrics should be recorded



In [30]:
# lr_finder = trainer.lr_find(model)
help(pl.Trainer)

Help on class Trainer in module pytorch_lightning.trainer.trainer:

class Trainer(pytorch_lightning.trainer.properties.TrainerProperties, pytorch_lightning.trainer.callback_hook.TrainerCallbackHookMixin, pytorch_lightning.trainer.model_hooks.TrainerModelHooksMixin, pytorch_lightning.trainer.optimizers.TrainerOptimizersMixin, pytorch_lightning.trainer.logging.TrainerLoggingMixin, pytorch_lightning.trainer.training_tricks.TrainerTrainingTricksMixin, pytorch_lightning.trainer.data_loading.TrainerDataLoadingMixin, pytorch_lightning.trainer.deprecated_api.DeprecatedDistDeviceAttributes, pytorch_lightning.trainer.deprecated_api.DeprecatedTrainerAttributes)
 |  Trainer(logger: Union[pytorch_lightning.loggers.base.LightningLoggerBase, Iterable[pytorch_lightning.loggers.base.LightningLoggerBase], bool] = True, checkpoint_callback: bool = True, callbacks: Union[List[pytorch_lightning.callbacks.base.Callback], pytorch_lightning.callbacks.base.Callback, NoneType] = None, default_root_dir: Union[st

In [22]:
filename = "muon_100k_set12_SRT.pt"
dataset, train_loader, test_loader, val_loader = fc.dataset_preparator(name = filename,
                                                                       path = path,
                                                                       transformer = transformer,
                                                                       tc = AzZe_target_constructor,
                                                                       fc = fc.custom_feature_constructor,
                                                                       shuffle = True,
                                                                       TrTV_split = (0,1,1),
                                                                       batch_size = args['batch_size'])

2021-02-19 13:58:07.486523: loading data..
2021-02-19 13:58:08.846916: executing target constructor..
2021-02-19 13:58:08.858912: executing feature constructor..
2021-02-19 13:58:17.235923: shuffling dataset..
2021-02-19 13:58:17.274909: defining dataloaders..
2021-02-19 13:58:17.275909: Done!


In [25]:
trainer.test(model,test_loader)


--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'Test Acc': nan}
--------------------------------------------------------------------------------


C:\Users\jv97\anaconda3\lib\site-packages\pytorch_lightning\utilities\distributed.py:50: UserWarning: The testing_epoch_end should not return anything as of 9.1. To log, use self.log(...) or self.write(...) directly in the LightningModule
  warnings.warn(*args, **kwargs)


[{'Test Acc': nan}]